# Challenge Details
You are expected to complete the following.
You will complete a pull request for the items and submit your work to this repository.
In addition, you will submit your .html/.pdf file in Canvas.

1. Select a latitude and longitude within your hometown (use the location of a family friend if you are not from the US).
    
    
    Latitude:   39.952583
    
    
    Longitude:  -75.165222

2. Pull data from the Daily Forecast 16-day API for the CBG for the next 16 days and print a snippet of the JSON file in your notebook.
3. Pull data from the history API for seven days in the past year for the same CBG and print a snippet of the JSON file in your notebook.
4. Store the information from each API request in a Spark DataFrame and display your dataframe in your notebook (see the image below and the table schema for examples of how your table should look).
5. Use the data from the created tables to create two visualizations (You can't use bar charts).
6. Use the data from the created tables to create two summary tables (tables you would see in a report).
7. A short document that explains the differences between REST and GraphQL APIs (2-3 paragraphs).

In [0]:
# imports
import pandas as pd

import requests
import json
import os
import datetime
import time

import pyspark.sql.functions as F

In [0]:
%run ./keys

### 1. Select a latitude and longitude within your hometown I chose Philadelphia for this project

In [0]:
# the above cell runs the keys file to use our api_key
# we separated so that clients don't see our api_key
api_key = ow_key

# Select a latitude and longitude within your hometown
# I chose Philadelphia for this project
lat = 39.952583
lon = -75.165222

### 2. Pull data from the Daily Forecast 16-day API for the CBG for the next 16 days and print a snippet of the JSON file in your notebook.

In [0]:
days = 16
days_url = f"https://api.openweathermap.org/data/2.5/forecast/daily?lat={lat}&lon={lon}&cnt={days}&appid={api_key}"

In [0]:
# response: 200 is good, 400 is bad
response = requests.get(days_url)
response

<Response [200]>

In [0]:
days_data = json.loads(response.text)
days_data

{'city': {'id': 4560349,
  'name': 'Philadelphia',
  'coord': {'lon': -75.1652, 'lat': 39.9526},
  'country': 'US',
  'population': 1526006,
  'timezone': -14400},
 'cod': '200',
 'message': 4.2569978,
 'cnt': 16,
 'list': [{'dt': 1697385600,
   'sunrise': 1697368235,
   'sunset': 1697408522,
   'temp': {'day': 288.35,
    'min': 282.54,
    'max': 289.02,
    'night': 283.17,
    'eve': 287.12,
    'morn': 283.26},
   'feels_like': {'day': 287.13,
    'night': 281.98,
    'eve': 286.22,
    'morn': 282.49},
   'pressure': 1002,
   'humidity': 46,
   'weather': [{'id': 501,
     'main': 'Rain',
     'description': 'moderate rain',
     'icon': '10d'}],
   'speed': 6.28,
   'deg': 318,
   'gust': 12.33,
   'clouds': 9,
   'pop': 1,
   'rain': 2.48},
  {'dt': 1697472000,
   'sunrise': 1697454698,
   'sunset': 1697494833,
   'temp': {'day': 285.24,
    'min': 281.12,
    'max': 288.12,
    'night': 283.48,
    'eve': 286.86,
    'morn': 281.12},
   'feels_like': {'day': 284.23,
    'night

In [0]:
# format the above json response to something readable
json_formatted_str = json.dumps(days_data, indent=3)
print(json_formatted_str)

{
   "city": {
      "id": 4560349,
      "name": "Philadelphia",
      "coord": {
         "lon": -75.1652,
         "lat": 39.9526
      },
      "country": "US",
      "population": 1526006,
      "timezone": -14400
   },
   "cod": "200",
   "message": 4.2569978,
   "cnt": 16,
   "list": [
      {
         "dt": 1697385600,
         "sunrise": 1697368235,
         "sunset": 1697408522,
         "temp": {
            "day": 288.35,
            "min": 282.54,
            "max": 289.02,
            "night": 283.17,
            "eve": 287.12,
            "morn": 283.26
         },
         "feels_like": {
            "day": 287.13,
            "night": 281.98,
            "eve": 286.22,
            "morn": 282.49
         },
         "pressure": 1002,
         "humidity": 46,
         "weather": [
            {
               "id": 501,
               "main": "Rain",
               "description": "moderate rain",
               "icon": "10d"
            }
         ],
         "speed": 6

In [0]:
# convert json response to pyspark dataframe
rdd = spark.sparkContext.parallelize([response.text])
days_df = spark.read.json(rdd)
display(days_df)

city,cnt,cod,list,message
"List(List(39.9526, -75.1652), US, 4560349, Philadelphia, 1526006, -14400)",16,200,"List(List(9, 318, 1697385600, List(287.13, 286.22, 282.49, 281.98), 12.33, 46, 1.0, 1002, 2.48, 6.28, 1697368235, 1697408522, List(288.35, 287.12, 289.02, 282.54, 283.26, 283.17), List(List(moderate rain, 10d, 501, Rain))), List(100, 302, 1697472000, List(284.23, 285.98, 278.87, 282.79), 10.11, 66, 0.37, 1005, 0.5, 3.86, 1697454698, 1697494833, List(285.24, 286.86, 288.12, 281.12, 281.12, 283.48), List(List(light rain, 10d, 500, Rain))), List(29, 331, 1697558400, List(287.71, 287.17, 280.6, 284.85), 6.99, 48, 0.11, 1013, null, 3.05, 1697541161, 1697581145, List(288.83, 288.15, 290.05, 281.5, 281.68, 285.66), List(List(scattered clouds, 03d, 802, Clouds))), List(100, 181, 1697644800, List(286.41, 290.62, 284.61, 285.52), 4.38, 62, 0.0, 1022, null, 1.96, 1697627625, 1697667457, List(287.32, 291.72, 291.72, 285.35, 285.52, 286.53), List(List(overcast clouds, 04d, 804, Clouds))), List(4, 196, 1697731200, List(290.1, 292.35, 283.8, 287.8), 11.93, 49, 0.0, 1024, null, 4.29, 1697714089, 1697753770, List(290.98, 293.1, 293.97, 284.66, 284.66, 288.37), List(List(sky is clear, 01d, 800, Clear))), List(100, 190, 1697817600, List(290.85, 290.79, 287.26, 287.75), 12.3, 64, 0.36, 1014, 0.24, 5.6, 1697800554, 1697840085, List(291.31, 291.09, 294.36, 287.78, 287.85, 287.8), List(List(light rain, 10d, 500, Rain))), List(100, 282, 1697904000, List(288.73, 288.53, 288.07, 285.21), 14.38, 91, 1.0, 995, 5.8, 6.74, 1697887019, 1697926400, List(288.74, 288.89, 290.67, 285.92, 288.04, 285.92), List(List(light rain, 10d, 500, Rain))), List(96, 288, 1697990400, List(284.97, 284.31, 284.0, 279.12), 11.43, 55, 0.01, 999, null, 6.59, 1697973484, 1698012717, List(286.18, 285.81, 286.86, 281.95, 284.87, 281.95), List(List(overcast clouds, 04d, 804, Clouds))), List(99, 306, 1698076800, List(280.25, 283.54, 278.35, 279.13), 10.22, 57, 0.0, 1013, null, 5.28, 1698059950, 1698099034, List(282.77, 285.14, 285.14, 281.07, 281.3, 281.07), List(List(overcast clouds, 04d, 804, Clouds))), List(0, 343, 1698163200, List(280.55, 284.78, 276.66, 279.55), 10.03, 47, 0.0, 1025, null, 5.46, 1698146416, 1698185353, List(283.06, 286.26, 286.32, 278.72, 278.81, 281.61), List(List(sky is clear, 01d, 800, Clear))), List(3, 30, 1698249600, List(283.48, 286.77, 277.59, 282.38), 8.87, 55, 0.0, 1032, null, 3.77, 1698232882, 1698271673, List(284.82, 288.03, 288.16, 279.76, 279.76, 283.54), List(List(sky is clear, 01d, 800, Clear))), List(0, 348, 1698336000, List(285.1, 289.08, 280.04, 283.71), 8.49, 51, 0.0, 1028, null, 3.57, 1698319349, 1698357993, List(286.39, 290.22, 290.39, 280.94, 281.57, 284.86), List(List(sky is clear, 01d, 800, Clear))), List(3, 323, 1698422400, List(287.89, 292.01, 281.42, 286.15), 8.95, 33, 0.0, 1016, null, 3.71, 1698405816, 1698444316, List(289.35, 292.98, 293.7, 282.52, 282.71, 287.13), List(List(sky is clear, 01d, 800, Clear))), List(3, 280, 1698508800, List(290.48, 293.97, 284.46, 288.9), 9.24, 52, 0.0, 1015, null, 3.55, 1698492283, 1698530639, List(291.26, 294.52, 295.26, 285.55, 285.55, 289.51), List(List(sky is clear, 01d, 800, Clear))), List(100, 336, 1698595200, List(287.94, 284.5, 287.56, 279.83), 11.39, 61, 0.87, 1016, 8.04, 5.58, 1698578751, 1698616964, List(288.73, 284.94, 288.73, 282.15, 288.22, 282.15), List(List(moderate rain, 10d, 501, Rain))), List(2, 322, 1698681600, List(279.94, 282.37, 276.03, 280.43), 11.5, 46, 0.27, 1027, null, 4.69, 1698665219, 1698703290, List(282.03, 284.24, 284.64, 278.5, 279.24, 281.16), List(List(sky is clear, 01d, 800, Clear))))",4.2569978


In [0]:
days_df.printSchema()

root
 |-- city: struct (nullable = true)
 |    |-- coord: struct (nullable = true)
 |    |    |-- lat: double (nullable = true)
 |    |    |-- lon: double (nullable = true)
 |    |-- country: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- population: long (nullable = true)
 |    |-- timezone: long (nullable = true)
 |-- cnt: long (nullable = true)
 |-- cod: string (nullable = true)
 |-- list: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- clouds: long (nullable = true)
 |    |    |-- deg: long (nullable = true)
 |    |    |-- dt: long (nullable = true)
 |    |    |-- feels_like: struct (nullable = true)
 |    |    |    |-- day: double (nullable = true)
 |    |    |    |-- eve: double (nullable = true)
 |    |    |    |-- morn: double (nullable = true)
 |    |    |    |-- night: double (nullable = true)
 |    |    |-- gust: double (nullable = true)
 |    |    |-- humidity: long (nul

### 3. Pull data from the history API for 7 days in the past year for the same CBG and print a snippet of the JSON file in your notebook

In [0]:
start = datetime.datetime(2023, 8, 23)
start = int(time.mktime(start.timetuple()))
end = datetime.datetime(2023, 8, 30)
end = int(time.mktime(end.timetuple()))
print(f"start: {start}")
print(f"end: {end}")
history_url = f"https://history.openweathermap.org/data/2.5/history/city?lat={lat}&lon={lon}&type=hour&start={start}&end={end}&appid={api_key}"

start: 1692748800
end: 1693353600


In [0]:
response = requests.get(history_url)
response

<Response [200]>

In [0]:
history_data = json.loads(response.text)
history_data

{'message': 'Count: 169',
 'cod': '200',
 'city_id': 1,
 'calctime': 0.073458075,
 'cnt': 169,
 'list': [{'dt': 1692748800,
   'main': {'temp': 295.92,
    'feels_like': 295.56,
    'pressure': 1019,
    'humidity': 50,
    'temp_min': 293.15,
    'temp_max': 297.98},
   'wind': {'speed': 2.91, 'deg': 22, 'gust': 7.36},
   'clouds': {'all': 99},
   'weather': [{'id': 804,
     'main': 'Clouds',
     'description': 'overcast clouds',
     'icon': '04n'}]},
  {'dt': 1692752400,
   'main': {'temp': 293.39,
    'feels_like': 293.06,
    'pressure': 1019,
    'humidity': 61,
    'temp_min': 290.15,
    'temp_max': 296.21},
   'wind': {'speed': 2.68, 'deg': 30, 'gust': 7.73},
   'clouds': {'all': 100},
   'weather': [{'id': 804,
     'main': 'Clouds',
     'description': 'overcast clouds',
     'icon': '04n'}]},
  {'dt': 1692756000,
   'main': {'temp': 292.37,
    'feels_like': 291.99,
    'pressure': 1019,
    'humidity': 63,
    'temp_min': 288.62,
    'temp_max': 295.21},
   'wind': {'spe

In [0]:
# format the above json response to something readable
json_formatted_str = json.dumps(history_data, indent=3)
print(json_formatted_str)

{
   "message": "Count: 169",
   "cod": "200",
   "city_id": 1,
   "calctime": 0.073458075,
   "cnt": 169,
   "list": [
      {
         "dt": 1692748800,
         "main": {
            "temp": 295.92,
            "feels_like": 295.56,
            "pressure": 1019,
            "humidity": 50,
            "temp_min": 293.15,
            "temp_max": 297.98
         },
         "wind": {
            "speed": 2.91,
            "deg": 22,
            "gust": 7.36
         },
         "clouds": {
            "all": 99
         },
         "weather": [
            {
               "id": 804,
               "main": "Clouds",
               "description": "overcast clouds",
               "icon": "04n"
            }
         ]
      },
      {
         "dt": 1692752400,
         "main": {
            "temp": 293.39,
            "feels_like": 293.06,
            "pressure": 1019,
            "humidity": 61,
            "temp_min": 290.15,
            "temp_max": 296.21
         },
         "wind

In [0]:
# convert json response to pyspark dataframe
rdd = spark.sparkContext.parallelize([response.text])
history_df = spark.read.json(rdd)
display(history_df)

calctime city_id cnt cod list message 0.073458075 1 169 200 List(List(List(99), 1692748800, List(295.56, 50, 1019, 295.92, 297.98, 293.15), null, List(List(overcast clouds, 04n, 804, Clouds)), List(22, 7.36, 2.91)), List(List(100), 1692752400, List(293.06, 61, 1019, 293.39, 296.21, 290.15), null, List(List(overcast clouds, 04n, 804, Clouds)), List(30, 7.73, 2.68)), List(List(76), 1692756000, List(291.99, 63, 1019, 292.37, 295.21, 288.62), null, List(List(broken clouds, 04n, 803, Clouds)), List(34, 7.51, 2.23)), List(List(64), 1692759600, List(291.03, 67, 1019, 291.4, 294.27, 287.62), null, List(List(broken clouds, 04n, 803, Clouds)), List(33, 4.76, 1.36)), List(List(51), 1692763200, List(290.24, 70, 1019, 290.61, 293.49, 286.62), null, List(List(broken clouds, 04n, 803, Clouds)), List(13, 3.03, 1.15)), List(List(42), 1692766800, List(289.72, 72, 1019, 290.09, 292.6, 286.62), null, List(List(scattered clouds, 03n, 802, Clouds)), List(2, 2.88, 1.24)), List(List(39), 1692770400, List(289.82, 74, 1020, 290.13, 292.43, 286.62), null, List(List(scattered clouds, 03n, 802, Clouds)), List(346, 2.89, 1.57)), List(List(0), 1692774000, List(289.86, 71, 1020, 290.24, 292.09, 286.66), null, List(List(clear sky, 01n, 800, Clear)), List(330, null, 3.09)), List(List(3), 1692777600, List(289.35, 76, 1019, 289.66, 291.87, 286.15), null, List(List(clear sky, 01n, 800, Clear)), List(331, 3.58, 0.89)), List(List(2), 1692781200, List(288.94, 78, 1019, 289.24, 291.27, 286.16), null, List(List(clear sky, 01n, 800, Clear)), List(234, 3.58, 0.45)), List(List(5), 1692784800, List(288.74, 81, 1020, 288.99, 291.27, 285.6), null, List(List(clear sky, 01n, 800, Clear)), List(220, 0.89, 0.45)), List(List(8), 1692788400, List(288.88, 81, 1021, 289.11, 291.51, 285.75), null, List(List(clear sky, 01d, 800, Clear)), List(14, 4.31, 1.38)), List(List(7), 1692792000, List(291.31, 72, 1020, 291.54, 293.27, 288.53), null, List(List(clear sky, 01d, 800, Clear)), List(0, 1.79, 0.45)), List(List(10), 1692795600, List(293.12, 64, 1020, 293.37, 295.25, 290.95), null, List(List(clear sky, 01d, 800, Clear)), List(36, 3.94, 2.49)), List(List(44), 1692799200, List(294.02, 62, 1020, 294.24, 296.21, 291.81), null, List(List(scattered clouds, 03d, 802, Clouds)), List(71, 3.58, 0.45)), List(List(62), 1692802800, List(295.05, 57, 1020, 295.29, 296.97, 293.48), null, List(List(broken clouds, 04d, 803, Clouds)), List(36, 3.2, 2.59)), List(List(72), 1692806400, List(296.92, 49, 1019, 297.18, 298.83, 295.39), null, List(List(broken clouds, 04d, 803, Clouds)), List(67, 2.68, 0.45)), List(List(77), 1692810000, List(297.8, 49, 1020, 297.98, 299.85, 296.41), null, List(List(broken clouds, 04d, 803, Clouds)), List(21, 2.26, 1.58)), List(List(76), 1692813600, List(298.83, 43, 1019, 299.06, 300.65, 297.05), null, List(List(broken clouds, 04d, 803, Clouds)), List(96, 4.47, 1.79)), List(List(75), 1692817200, List(299.56, 43, 1018, 299.56, 301.21, 297.46), null, List(List(broken clouds, 04d, 803, Clouds)), List(140, null, 2.06)), List(List(83), 1692820800, List(299.13, 45, 1018, 299.13, 300.76, 296.8), null, List(List(broken clouds, 04d, 803, Clouds)), List(0, 2.68, 0.45)), List(List(83), 1692824400, List(298.45, 47, 1017, 298.62, 300.21, 296.35), null, List(List(broken clouds, 04d, 803, Clouds)), List(6, 1.61, 1.27)), List(List(87), 1692828000, List(297.9, 50, 1016, 298.05, 299.32, 295.8), null, List(List(overcast clouds, 04d, 804, Clouds)), List(26, 1.73, 1.22)), List(List(90), 1692831600, List(296.99, 54, 1016, 297.13, 298.71, 294.69), null, List(List(overcast clouds, 04d, 804, Clouds)), List(67, 2.76, 1.65)), List(List(89), 1692835200, List(295.72, 62, 1015, 295.78, 297.38, 293.58), null, List(List(overcast clouds, 04n, 804, Clouds)), List(0, 4.92, 0.45)), List(List(100), 1692838800, List(295.03, 65, 1017, 295.08, 296.51, 292.92), null, List(List(overcast clouds, 04n, 804, Clouds)), List(132, 6.26, 1.79)), List(List(96), 1692842400, List(294.48, 65, 1017, 294.58, 295.68, 292.97), null, List

### 4. Store the information from each API request in a Spark DataFrame and display your dataframe in your notebook (see the image above and the table schema for examples of how your table should look).

In [0]:
# 4. Store the information from each API request in a Spark DataFrame 
# and display your dataframe in your notebook 
# (see the image above and the table schema for examples of how your table should look).
days_table = days_df\
    .select("city.*", "city.coord.*", "*")\
    .withColumn("item", F.explode(F.col("list")))\
    .drop("list")\
    .select("*", "item.*")\
    .select(
        "*",
        F.col("feels_like.day").alias("feels_like_day"),
        F.col("feels_like.eve").alias("feels_like_eve"),
        F.col("feels_like.morn").alias("feels_like_morn"),
        F.col("feels_like.night").alias("feels_like_night"),
        F.col("temp.day").alias("temp_day"),
        F.col("temp.eve").alias("temp_eve"),
        F.col("temp.morn").alias("temp_morn"),
        F.col("temp.night").alias("temp_night"),
        F.col("temp.min").alias("temp_min"),
        F.col("temp.max").alias("temp_max"),
    )\
    .drop("item")\
    .select([
        'id',
        'name',
        'lat',
        'lon',
        'timezone',
        'population',
        'dt',
        'sunrise',
        'sunset',
        'pop',
        'speed',
        'gust',
        'deg',
        'clouds',
        'pressure',
        'rain',
        'temp_min',
        'temp_max',
        'temp_morn',
        'temp_day',
        'temp_eve',
        'temp_night',
        'feels_like_morn',
        'feels_like_day',
        'feels_like_eve',
        'feels_like_night',
        'weather'
    ])
    
display(days_table)

id,name,lat,lon,timezone,population,dt,sunrise,sunset,pop,speed,gust,deg,clouds,pressure,rain,temp_min,temp_max,temp_morn,temp_day,temp_eve,temp_night,feels_like_morn,feels_like_day,feels_like_eve,feels_like_night,weather
4560349,Philadelphia,39.9526,-75.1652,-14400,1526006,1697385600,1697368235,1697408522,1.0,6.28,12.33,318,9,1002,2.48,282.54,289.02,283.26,288.35,287.12,283.17,282.49,287.13,286.22,281.98,"List(List(moderate rain, 10d, 501, Rain))"
4560349,Philadelphia,39.9526,-75.1652,-14400,1526006,1697472000,1697454698,1697494833,0.37,3.86,10.11,302,100,1005,0.5,281.12,288.12,281.12,285.24,286.86,283.48,278.87,284.23,285.98,282.79,"List(List(light rain, 10d, 500, Rain))"
4560349,Philadelphia,39.9526,-75.1652,-14400,1526006,1697558400,1697541161,1697581145,0.11,3.05,6.99,331,29,1013,null,281.5,290.05,281.68,288.83,288.15,285.66,280.6,287.71,287.17,284.85,"List(List(scattered clouds, 03d, 802, Clouds))"
4560349,Philadelphia,39.9526,-75.1652,-14400,1526006,1697644800,1697627625,1697667457,0.0,1.96,4.38,181,100,1022,null,285.35,291.72,285.52,287.32,291.72,286.53,284.61,286.41,290.62,285.52,"List(List(overcast clouds, 04d, 804, Clouds))"
4560349,Philadelphia,39.9526,-75.1652,-14400,1526006,1697731200,1697714089,1697753770,0.0,4.29,11.93,196,4,1024,null,284.66,293.97,284.66,290.98,293.1,288.37,283.8,290.1,292.35,287.8,"List(List(sky is clear, 01d, 800, Clear))"
4560349,Philadelphia,39.9526,-75.1652,-14400,1526006,1697817600,1697800554,1697840085,0.36,5.6,12.3,190,100,1014,0.24,287.78,294.36,287.85,291.31,291.09,287.8,287.26,290.85,290.79,287.75,"List(List(light rain, 10d, 500, Rain))"
4560349,Philadelphia,39.9526,-75.1652,-14400,1526006,1697904000,1697887019,1697926400,1.0,6.74,14.38,282,100,995,5.8,285.92,290.67,288.04,288.74,288.89,285.92,288.07,288.73,288.53,285.21,"List(List(light rain, 10d, 500, Rain))"
4560349,Philadelphia,39.9526,-75.1652,-14400,1526006,1697990400,1697973484,1698012717,0.01,6.59,11.43,288,96,999,null,281.95,286.86,284.87,286.18,285.81,281.95,284.0,284.97,284.31,279.12,"List(List(overcast clouds, 04d, 804, Clouds))"
4560349,Philadelphia,39.9526,-75.1652,-14400,1526006,1698076800,1698059950,1698099034,0.0,5.28,10.22,306,99,1013,null,281.07,285.14,281.3,282.77,285.14,281.07,278.35,280.25,283.54,279.13,"List(List(overcast clouds, 04d, 804, Clouds))"
4560349,Philadelphia,39.9526,-75.1652,-14400,1526006,1698163200,1698146416,1698185353,0.0,5.46,10.03,343,0,1025,null,278.72,286.32,278.81,283.06,286.26,281.61,276.66,280.55,284.78,279.55,"List(List(sky is clear, 01d, 800, Clear))"


In [0]:
history_table = history_df\
    .join(
        days_df.select(
            F.col("city.coord.lat").alias("lat"),
            F.col("city.coord.lon").alias("lon")
        ),
        how="inner"
    )\
    .withColumn("item", F.explode(F.col("list")))\
    .drop("list", "calctime", "cnt", "cod", "message")\
    .select("*", "item.*")\
    .select(
        "*",
        "main.*",
        F.col("city_id").alias("id"),
        F.col("clouds.all").alias("clouds_all"),
        F.col("rain.1h").alias("rain_1h"),
        F.col("wind.speed").alias("wind_speed"),
        F.col("wind.gust").alias("wind_gust"),
        F.col("wind.deg").alias("wind_deg")
    )\
    .select([
        'id',
        'lat',
        'lon',
        'dt',
        'wind_speed',
        'wind_gust',
        'wind_deg',
        'clouds_all',
        'pressure',
        'rain_1h',
        'temp_min',
        'temp_max',
        'temp',
        'weather'
    ])

display(history_table)

id,lat,lon,dt,wind_speed,wind_gust,wind_deg,clouds_all,pressure,rain_1h,temp_min,temp_max,temp,weather
1,39.9526,-75.1652,1692748800,2.91,7.36,22,99,1019,null,293.15,297.98,295.92,"List(List(overcast clouds, 04n, 804, Clouds))"
1,39.9526,-75.1652,1692752400,2.68,7.73,30,100,1019,null,290.15,296.21,293.39,"List(List(overcast clouds, 04n, 804, Clouds))"
1,39.9526,-75.1652,1692756000,2.23,7.51,34,76,1019,null,288.62,295.21,292.37,"List(List(broken clouds, 04n, 803, Clouds))"
1,39.9526,-75.1652,1692759600,1.36,4.76,33,64,1019,null,287.62,294.27,291.4,"List(List(broken clouds, 04n, 803, Clouds))"
1,39.9526,-75.1652,1692763200,1.15,3.03,13,51,1019,null,286.62,293.49,290.61,"List(List(broken clouds, 04n, 803, Clouds))"
1,39.9526,-75.1652,1692766800,1.24,2.88,2,42,1019,null,286.62,292.6,290.09,"List(List(scattered clouds, 03n, 802, Clouds))"
1,39.9526,-75.1652,1692770400,1.57,2.89,346,39,1020,null,286.62,292.43,290.13,"List(List(scattered clouds, 03n, 802, Clouds))"
1,39.9526,-75.1652,1692774000,3.09,null,330,0,1020,null,286.66,292.09,290.24,"List(List(clear sky, 01n, 800, Clear))"
1,39.9526,-75.1652,1692777600,0.89,3.58,331,3,1019,null,286.15,291.87,289.66,"List(List(clear sky, 01n, 800, Clear))"
1,39.9526,-75.1652,1692781200,0.45,3.58,234,2,1019,null,286.16,291.27,289.24,"List(List(clear sky, 01n, 800, Clear))"


### 5. Use the data from the created tables to create two visualizations (You can't use bar charts).

In [0]:
display(days_table)

id,name,lat,lon,timezone,population,dt,sunrise,sunset,pop,speed,gust,deg,clouds,pressure,rain,temp_min,temp_max,temp_morn,temp_day,temp_eve,temp_night,feels_like_morn,feels_like_day,feels_like_eve,feels_like_night,weather
4560349,Philadelphia,39.9526,-75.1652,-14400,1526006,1697385600,1697368235,1697408522,1.0,6.28,12.33,318,9,1002,2.48,282.54,289.02,283.26,288.35,287.12,283.17,282.49,287.13,286.22,281.98,"List(List(moderate rain, 10d, 501, Rain))"
4560349,Philadelphia,39.9526,-75.1652,-14400,1526006,1697472000,1697454698,1697494833,0.37,3.86,10.11,302,100,1005,0.5,281.12,288.12,281.12,285.24,286.86,283.48,278.87,284.23,285.98,282.79,"List(List(light rain, 10d, 500, Rain))"
4560349,Philadelphia,39.9526,-75.1652,-14400,1526006,1697558400,1697541161,1697581145,0.11,3.05,6.99,331,29,1013,null,281.5,290.05,281.68,288.83,288.15,285.66,280.6,287.71,287.17,284.85,"List(List(scattered clouds, 03d, 802, Clouds))"
4560349,Philadelphia,39.9526,-75.1652,-14400,1526006,1697644800,1697627625,1697667457,0.0,1.96,4.38,181,100,1022,null,285.35,291.72,285.52,287.32,291.72,286.53,284.61,286.41,290.62,285.52,"List(List(overcast clouds, 04d, 804, Clouds))"
4560349,Philadelphia,39.9526,-75.1652,-14400,1526006,1697731200,1697714089,1697753770,0.0,4.29,11.93,196,4,1024,null,284.66,293.97,284.66,290.98,293.1,288.37,283.8,290.1,292.35,287.8,"List(List(sky is clear, 01d, 800, Clear))"
4560349,Philadelphia,39.9526,-75.1652,-14400,1526006,1697817600,1697800554,1697840085,0.36,5.6,12.3,190,100,1014,0.24,287.78,294.36,287.85,291.31,291.09,287.8,287.26,290.85,290.79,287.75,"List(List(light rain, 10d, 500, Rain))"
4560349,Philadelphia,39.9526,-75.1652,-14400,1526006,1697904000,1697887019,1697926400,1.0,6.74,14.38,282,100,995,5.8,285.92,290.67,288.04,288.74,288.89,285.92,288.07,288.73,288.53,285.21,"List(List(light rain, 10d, 500, Rain))"
4560349,Philadelphia,39.9526,-75.1652,-14400,1526006,1697990400,1697973484,1698012717,0.01,6.59,11.43,288,96,999,null,281.95,286.86,284.87,286.18,285.81,281.95,284.0,284.97,284.31,279.12,"List(List(overcast clouds, 04d, 804, Clouds))"
4560349,Philadelphia,39.9526,-75.1652,-14400,1526006,1698076800,1698059950,1698099034,0.0,5.28,10.22,306,99,1013,null,281.07,285.14,281.3,282.77,285.14,281.07,278.35,280.25,283.54,279.13,"List(List(overcast clouds, 04d, 804, Clouds))"
4560349,Philadelphia,39.9526,-75.1652,-14400,1526006,1698163200,1698146416,1698185353,0.0,5.46,10.03,343,0,1025,null,278.72,286.32,278.81,283.06,286.26,281.61,276.66,280.55,284.78,279.55,"List(List(sky is clear, 01d, 800, Clear))"


Databricks visualization. Run in Databricks to view.

In [0]:
display(history_table)

id,lat,lon,dt,wind_speed,wind_gust,wind_deg,clouds_all,pressure,rain_1h,temp_min,temp_max,temp,weather
1,39.9526,-75.1652,1692748800,2.91,7.36,22,99,1019,null,293.15,297.98,295.92,"List(List(overcast clouds, 04n, 804, Clouds))"
1,39.9526,-75.1652,1692752400,2.68,7.73,30,100,1019,null,290.15,296.21,293.39,"List(List(overcast clouds, 04n, 804, Clouds))"
1,39.9526,-75.1652,1692756000,2.23,7.51,34,76,1019,null,288.62,295.21,292.37,"List(List(broken clouds, 04n, 803, Clouds))"
1,39.9526,-75.1652,1692759600,1.36,4.76,33,64,1019,null,287.62,294.27,291.4,"List(List(broken clouds, 04n, 803, Clouds))"
1,39.9526,-75.1652,1692763200,1.15,3.03,13,51,1019,null,286.62,293.49,290.61,"List(List(broken clouds, 04n, 803, Clouds))"
1,39.9526,-75.1652,1692766800,1.24,2.88,2,42,1019,null,286.62,292.6,290.09,"List(List(scattered clouds, 03n, 802, Clouds))"
1,39.9526,-75.1652,1692770400,1.57,2.89,346,39,1020,null,286.62,292.43,290.13,"List(List(scattered clouds, 03n, 802, Clouds))"
1,39.9526,-75.1652,1692774000,3.09,null,330,0,1020,null,286.66,292.09,290.24,"List(List(clear sky, 01n, 800, Clear))"
1,39.9526,-75.1652,1692777600,0.89,3.58,331,3,1019,null,286.15,291.87,289.66,"List(List(clear sky, 01n, 800, Clear))"
1,39.9526,-75.1652,1692781200,0.45,3.58,234,2,1019,null,286.16,291.27,289.24,"List(List(clear sky, 01n, 800, Clear))"


Databricks visualization. Run in Databricks to view.

### 6. Use the data from the created tables to create two summary tables (tables you would see in a report).

In [0]:
days_summary = days_table\
    .summary()\

display(days_summary)

summary,id,name,lat,lon,timezone,population,dt,sunrise,sunset,pop,speed,gust,deg,clouds,pressure,rain,temp_min,temp_max,temp_morn,temp_day,temp_eve,temp_night,feels_like_morn,feels_like_day,feels_like_eve,feels_like_night
count,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,5,16,16,16,16,16,16,16,16,16,16
mean,4560349.0,null,39.9526,-75.1652,-14400.0,1526006.0,1.6980336E9,1.6980167206875E9,1.6980558869375E9,0.249375,4.62375,10.158750000000001,273.5,46.75,1015.375,3.412,282.50187500000004,289.819375,283.385,287.21,288.6918749999999,284.619375,281.98812499999997,285.98499999999996,287.686875,283.4437500000001
stddev,0.0,null,0.0,0.0,0.0,0.0,411346.2774840682,411658.7289952598,410954.65429065254,0.3753659325689178,1.3707364200798537,2.3860004889633477,84.615207459022,48.316318844326986,10.849731179465575,3.4106773520812546,2.674213202046539,3.3107148648995337,3.1140648676609195,3.007625863257159,3.228731884708499,2.7361395401794337,3.9040973668698324,3.562768586366509,3.48754109892152,3.2904100554996525
min,4560349,Philadelphia,39.9526,-75.1652,-14400,1526006,1697385600,1697368235,1697408522,0.0,1.96,4.38,30,0,995,0.24,278.5,284.64,278.81,282.03,284.24,281.07,276.03,279.94,282.37,279.12
25%,4560349,null,39.9526,-75.1652,-14400,1526006,1697644800,1697627625,1697667457,0.0,3.57,8.87,196,3,1005,0.5,280.94,286.86,281.12,284.82,285.81,281.95,278.35,283.48,284.5,279.83
50%,4560349,null,39.9526,-75.1652,-14400,1526006,1697990400,1697973484,1698012717,0.0,4.29,10.11,302,9,1015,2.48,281.95,289.02,282.71,287.32,288.03,283.54,281.42,286.41,286.77,282.79
75%,4560349,null,39.9526,-75.1652,-14400,1526006,1698336000,1698319349,1698357993,0.36,5.58,11.5,323,100,1024,5.8,284.66,291.72,285.52,288.83,291.09,286.53,284.46,287.94,290.62,285.52
max,4560349,Philadelphia,39.9526,-75.1652,-14400,1526006,1698681600,1698665219,1698703290,1.0,6.74,14.38,348,100,1032,8.04,287.78,295.26,288.22,291.31,294.52,289.51,288.07,290.85,293.97,288.9


In [0]:
history_summary = history_table\
  .summary()

display(history_summary)

summary,id,lat,lon,dt,wind_speed,wind_gust,wind_deg,clouds_all,pressure,rain_1h,temp_min,temp_max,temp
count,169,169,169,169,169,123,169,169,169,3,169,169,169
mean,1.0,39.95259999999988,-75.1651999999999,1.6930512E9,1.914437869822484,4.1959349593495965,112.44378698224853,74.06508875739645,1013.5207100591716,0.37666666666666665,294.13201183431954,297.9169822485208,296.2731360946749
stddev,0.0,0.0,0.0,176148.80073392496,1.2556604494335346,1.8245380450136552,90.82445509176623,32.00542030207248,3.3257450962447614,0.04041451884327378,3.4649698651684324,3.239357814946439,3.2385007534625774
min,1,39.9526,-75.1652,1692748800,0.0,0.89,0,0,1008,0.34,285.6,291.27,288.99
25%,1,39.9526,-75.1652,1692900000,0.89,3.02,36,49,1011,0.34,292.14,295.61,294.16
50%,1,39.9526,-75.1652,1693051200,1.65,4.02,98,90,1014,0.37,293.61,297.07,295.54
75%,1,39.9526,-75.1652,1693202400,2.57,4.92,156,100,1015,0.42,296.61,300.13,298.55
max,1,39.9526,-75.1652,1693353600,6.69,11.02,352,100,1021,0.42,300.92,304.84,302.94


### 7. A short document that explains the differences between REST and GraphQL APIs (2-3 paragraphs).

Rest API and GraphQL API are two different Application Programming Interface (API) used for data exchange over the internet. In layman's term, Rest API and GraphQL API are both differently structured ways of sending and getting data from one point of the internet to another. Rest API uses HTTP verbs to send and receive data. The HTTP verbs are: POST, GET, PUT, and DELETE. POST is used to create a new resource. GET is used to take something from a resource. PUT is used to update a current resource or create a new resource. DELETE is used to remove something from a resource. Using these verbs in the Rest API, one can do tasks like getting the follower counts of a user from a social media site by two GET commands. The first to get the id, then the second to get the follower count of the user with that id. Not only is this tiresome, but it is inconsistent. Sometimes the Rest API over-fetches info, and other time under-fetches info. GraphQL was created to eliminate all those extra commands without over or under-fetching. With GraphQL, we’d only have to do one HTTP Post command with exactly the user ID and followers to get the same data.


Both Rest API and GraphQL API get info by using a get command with a URL. Once able to get the info, then both APIs can return a JSON object back to the site that requested the data. The difference lies in how it gets the data, as mentioned in the previous paragraph, and how the server determines the resources. Since the endpoint is the identity of the data that you requested in the Rest API, the server determines the shape and size of the resource. In GraphQL, on the other hand, the identity of the data is not its endpoint, and we can request for specific part of the data. This means that the server doesn’t determine the size of the data, the user request does. The server only determines what resources are available to get using GraphQL.